In [ ]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [ ]:
def api_call_func(api_params: dict) -> list[dict]:

    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    fields_purchase_lines = ['product_id', 'price_unit', 'date_order', 'order_id']
    purchase_lines_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("date_order", ">=", (datetime.today() - timedelta(days = 100)).strftime("%Y-%m-%d"))]])
    purchase_lines_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [purchase_lines_ids], {'fields':fields_purchase_lines})


    fields_product_template = ['name', 'type', 'standard_price', 'list_price', 'write_date']
    product_template_ids = models.execute_kw(api_db, uid, api_clave, 'product.template', 'search', [[]])
    product_template_json = models.execute_kw(api_db, uid, api_clave, 'product.template', 'read', [product_template_ids], {'fields':fields_product_template})


    return purchase_lines_json, product_template_json

In [ ]:
def purchase_lines_df_fun(purchase_lines_json: list[dict]) -> pd.DataFrame:

    data_purchase_lines = []

    for purchase in purchase_lines_json:
        new = {}
        new['purchase_line_id'] = purchase['id']
        new['product_id'] = purchase['product_id'][0]
        new['purchase_cost_unit'] = purchase['price_unit']
        new['purchase_date_order'] = purchase['date_order']
        new['purchase_order_id'] = purchase['order_id'][0]

        data_purchase_lines.append(new)

    purchase_lines_df1 = pd.DataFrame(data_purchase_lines)
    purchase_lines_df1['purchase_date_order'] = pd.to_datetime(purchase_lines_df1['purchase_date_order'], format='%Y-%m-%d %H:%M:%S')
    purchase_lines_df1['purchase_line_id'] = purchase_lines_df1['purchase_line_id'].astype('Int64')
    purchase_lines_df1['purchase_order_id'] = purchase_lines_df1['purchase_order_id'].astype('Int64')

    purchase_lines_df = purchase_lines_df1.loc[purchase_lines_df1['product_id'].drop_duplicates().index]

    return purchase_lines_df

In [ ]:
def product_template_df_fun(product_template_json: list[dict]) -> pd.DataFrame:
    
    data_product_template = []

    for product in product_template_json:
        new = {}
        new['product_id'] = product['id']
        new['product_type'] = product['type']
        new['product_description'] = product['name']
        new['product_template_cost'] = product['standard_price']
        new['product_price'] = product['list_price']
        new['write_date'] = product['write_date']

        data_product_template.append(new)

    product_template_df = pd.DataFrame(data_product_template)
    product_template_df['write_date'] = pd.to_datetime(product_template_df['write_date'], format='%Y-%m-%d %H:%M:%S')
    product_template_df['product_id'] = product_template_df['product_id'].astype('Int64')

    return product_template_df

In [ ]:
def complete_costo_df_fun(purchase_lines_df: pd.DataFrame, product_template_df: pd.DataFrame) -> pd.DataFrame:
    
    complete_costo_df = product_template_df.merge(purchase_lines_df, how='left', on='product_id')


    complete_costo_df.loc[(complete_costo_df['write_date'] > complete_costo_df['purchase_date_order']) | (complete_costo_df['purchase_date_order'].isna()), 'cost_type'] = 'REPS'
    complete_costo_df.loc[(complete_costo_df['write_date'] > complete_costo_df['purchase_date_order']) | (complete_costo_df['purchase_date_order'].isna()), 'cost_unit'] = complete_costo_df['product_template_cost']
    
    complete_costo_df.loc[complete_costo_df['purchase_date_order'] > complete_costo_df['write_date'], 'cost_type'] = 'Compra'
    complete_costo_df.loc[complete_costo_df['purchase_date_order'] > complete_costo_df['write_date'], 'cost_unit'] = complete_costo_df['purchase_cost_unit']
    
    complete_costo_df.loc[complete_costo_df['product_type'] == 'service', 'cost_type'] = 'Servicio'
    complete_costo_df.loc[complete_costo_df['product_type'] == 'service', 'cost_unit'] = pd.NA

    complete_costo_df['cost_unit'] = complete_costo_df['cost_unit'] * -1


    return complete_costo_df


In [ ]:
def costo_func(test_db: bool = False) -> pd.DataFrame:
    
    api_params = api_params_func(test_db)
    purchase_lines_json, product_template_json = api_call_func(api_params)

    purchase_lines_df = purchase_lines_df_fun(purchase_lines_json)
    product_template_df = product_template_df_fun(product_template_json)


    complete_costo_df = complete_costo_df_fun(purchase_lines_df, product_template_df)

    return complete_costo_df


## Pruebas

In [ ]:
api_params = api_params_func()
purchase_lines_json, product_template_json = api_call_func(api_params)

purchase_lines_df = purchase_lines_df_fun(purchase_lines_json)
product_template_df = product_template_df_fun(product_template_json)

complete_costo_df = complete_costo_df = complete_costo_df_fun(purchase_lines_df, product_template_df)
